In [49]:
#Import Libraries
import pandas as pd
import openpyxl
from random import randint, choice, shuffle

In [50]:
#Name of the Excel file and sheets to be used
file_name = r'SecretSantaDataSet.xlsx'
people_sheet = r'Pessoas'
incompatibles_sheet = r'Incompatibilidades'

In [51]:
#Import participants from excel and put them on a list
people = pd.read_excel(file_name, engine='openpyxl', sheet_name=people_sheet)
people = list(people.loc[:, 'Nome da Pessoa'])

In [52]:
#Import incompatibilities between participants and put them on a list of sets
incompatibilities = pd.read_excel(file_name, engine='openpyxl', sheet_name=incompatibles_sheet)
incompats = []
for index, row in incompatibilities.iterrows():
    a_set = {row['Nome da Pessoa 1'], row['Nome da Pessoa 2']}
    incompats.append(a_set)

In [53]:
#Alternativa usando openpyxl para abrir e ler o workbook
""" wb = openpyxl.load_workbook(file_name)
ws_people = wb[people_sheet]
ws_incompats = wb[incompatibles_sheet]

data_people = ws_people.values
columns_people = next(data_people)
df_people = pd.DataFrame(data_people, columns=columns_people)
people = list(df_people.loc[:, 'Nome da Pessoa'])

data_incompats = ws_incompats.values
columns_incompats = next(data_incompats)
df_incompats = pd.DataFrame(data_incompats, columns=columns_incompats)
incompats = []
for index, row in df_incompats.iterrows():
    a_set = {row['Nome da Pessoa 2'], row['Nome da Pessoa 2']}
    incompats.append(a_set) """

" wb = openpyxl.load_workbook(file_name)\nws_people = wb[people_sheet]\nws_incompats = wb[incompatibles_sheet]\n\ndata_people = ws_people.values\ncolumns_people = next(data_people)\ndf_people = pd.DataFrame(data_people, columns=columns_people)\npeople = list(df_people.loc[:, 'Nome da Pessoa'])\n\ndata_incompats = ws_incompats.values\ncolumns_incompats = next(data_incompats)\ndf_incompats = pd.DataFrame(data_incompats, columns=columns_incompats)\nincompats = []\nfor index, row in df_incompats.iterrows():\n    a_set = {row['Nome da Pessoa 2'], row['Nome da Pessoa 2']}\n    incompats.append(a_set) "

In [54]:
#FUNCTION FIND_INCOMPATIBILITIES
#Returns a set of people that a "name_set" cannot match with, based on the incompats list
#Returns  subset of the incompats list for a person
def find_incompatibilities(name_set, exclusion_list):
    incompatibles = set()
    for i in range(len(exclusion_list)):
        if name_set.issubset(exclusion_list[i]):
            incompatibles = incompatibles.union(exclusion_list[i])
    return incompatibles

In [55]:
#FUNCTION FIND_POSSIBILITIES
#Returns the list of possible matches for a person, excluding themselves
#Returns the list of the difference between all participants and the incompatibilities for a person
def find_possibilities(name_set, receivers_set, exclusion_list):
    possibilities_set = set()
    possibilities_set = receivers_set.difference(name_set)
    possibilities_set = possibilities_set.difference(find_incompatibilities(name_set, exclusion_list))
    return list(possibilities_set)

In [63]:
#Sorting hat
max_tries = 2
tries = 0
while tries < max_tries:
    shuffle(people)
    givers = people.copy()
    receivers = people.copy()
    matches = []
    for p in people :
        #Gets a list of all possible receivers for a participant p
        all_possible_rec = find_possibilities({givers[0]}, set(receivers), incompats)
        if len(all_possible_rec) == 0 :
            break
        #Selects a match from all possible receivers
        match = all_possible_rec[randint(0, len(all_possible_rec)-1)]
        matches.append([givers[0], match])
        givers.pop(0)
        receivers.pop(receivers.index(match))
    tries += 1
    if len(matches) == len(people):
        break

In [ ]:
print(f'{tries} tries; {len(matches)} out of {len(people)} matches: {matches}')

if tries == max_tries and len(matches) != len(people):
    print(f'Maybe try again next year with more participants and less restrictions.')

In [71]:
df_matches = pd.DataFrame(matches, columns=['Gift Giver', 'Gift Receiver'])
df_matches.loc[3, :]

Gift Giver       RITA ISABEL LARANJEIRA COSTA RIBEIRO
Gift Receiver           ANA RITA LOPES BORGES MARTINS
Name: 3, dtype: object

In [87]:
wb = openpyxl.load_workbook(file_name)
ws_new = wb.create_sheet("Resultados")
ws_new.append(list(df_matches.columns))
for i in range(len(df_matches)):
    ws_new.append(list(df_matches.loc[i, :]))
ws_new.column_dimensions['A'].width = 55
ws_new.column_dimensions['B'].width = 55
wb.save(file_name)
#wb.save(file_name)
#wb.close()


In [89]:
wb.remove(ws_new)

In [ ]:
wb.save(file_name)
wb.close()